In [1]:
import os

In [ ]:
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['ACTIVELOOP_TOKEN'] = os.getenv("ACTIVELOOP_TOKEN")

In [ ]:
from pytube import YouTube

In [ ]:
link = 'enter link here'

In [ ]:
try: 
    yt = YouTube(link) 
except: 
    print("Connection Error")

In [ ]:
yt.streams.filter(file_extension='mp4')
stream = yt.streams.get_by_itag()
stream.download('',"video.mp4")


In [ ]:
import whisper

model = whisper.load_model("base")
result = model.transcribe("video.mp4")
print(result['text'])

In [ ]:
with open ('text.txt', 'w') as file:  
    file.write(result['text'])

In [ ]:
from langchain import OpenAI, LLMChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate
from langchain.chains.summarize import load_summarize_chain

In [ ]:
llm = OpenAI(model_name="text-davinci-003", temperature=0)


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=0, separators=[" ", ",", "\n"]
)

In [ ]:
from langchain.docstore.document import Document

with open('text.txt') as f:
    text = f.read()

texts = text_splitter.split_text(text)
docs = [Document(page_content=t) for t in texts[:4]]

In [ ]:
from langchain.chains.summarize import load_summarize_chain
import textwrap

chain = load_summarize_chain(llm, chain_type="map_reduce")

output_summary = chain.run(docs)
wrapped_text = textwrap.fill(output_summary, width=100)
print(wrapped_text)

In [ ]:
print( chain.llm_chain.prompt.template )

prompt_template = """Write a concise bullet point summary of the following:


{text}


CONSCISE SUMMARY IN BULLET POINTS:"""

BULLET_POINT_PROMPT = PromptTemplate(template=prompt_template, 
                        input_variables=["text"])

In [ ]:
chain = load_summarize_chain(llm, 
                             chain_type="stuff", 
                             prompt=BULLET_POINT_PROMPT)

output_summary = chain.run(docs)

wrapped_text = textwrap.fill(output_summary, 
                             width=1000,
                             break_long_words=False,
                             replace_whitespace=False)
print(wrapped_text)

In [ ]:
from langchain.vectorstores import DeepLake
from langchain.embeddings.openai import OpenAIEmbeddings

In [ ]:
embeddings = OpenAIEmbeddings(model='text-embedding-ada-002')
